### Guia de ejercicios spark

In [1]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [ ]:
# Notas alumnos: (padron, materia, nota, fecha)
# Cuantos alumnos aprobaron al menos 1 materia en los ultimos 2 años
# Un rdd conteniendo el promedio de notas de cada alumno (padron promedio)
# El nombre y apellido del mejor promedio. rdd=(padron, nombre, apellido)

In [15]:
notas = [
    (100016, 1, 8, 20200218),
    (100011, 1, 1, 20200218),
    (100012, 1, 2, 20200218),
    (100013, 1, 6, 20180218),
    (100016, 1, 5, 20200218),
    (100014, 1, 10, 20200218),
    (100011, 1, 4, 20190218),
    (100011, 1, 9, 20160218),
    (100012, 1, 2, 20200218),
    ]

In [53]:
alumnos = [
    (100016,"eliana", "gamarra"),
    (100011,"pepe", "mujica"),
    (100012,"aldo", "aldepo"),
    (100014,"camila", "jasto"),
    (100013,"macri", "turro"),
    
]

rdd_alumnos = sc.parallelize(alumnos)

In [16]:
rrd_notas = sc.parallelize(notas)

In [20]:
aprobado_1 = rrd_notas.filter(lambda x: x[3] > 20181231)\
                    .map(lambda x: (x[0], x[2] > 3))\
                    .reduceByKey(lambda x,y: x|y)\
                    .filter(lambda x: x[1] != False)\
                    .count()
    
aprobado_1

3

In [23]:
prom = rrd_notas.map(lambda x: (x[0], x[2]))\
                .groupByKey()\
                .map(lambda x: (x[0], 
                sum(map(lambda y:y+y, list(x[1])))/len(list(x[1]))))
             
prom.collect()             

[(100016, 13.0),
 (100012, 4.0),
 (100013, 12.0),
 (100014, 20.0),
 (100011, 9.333333333333334)]

In [32]:
prom_2 = rrd_notas.map(lambda x: (x[0], (x[2],1)))\
                .reduceByKey(lambda x,y: (x[0]+ y[0], x[1]+y[1]))\
                .map(lambda x: (x[0], x[1][0] /x[1][1]))
                             
prom_2.collect()

[(100016, 6.5),
 (100012, 2.0),
 (100013, 6.0),
 (100014, 10.0),
 (100011, 4.666666666666667)]

In [60]:
alumns = rdd_alumnos.map(lambda x: (x[0], (x[1],x[2])))
mejor = prom_2.join(alumns)\
        .sortBy(lambda x: -x[1][0])\
        .map(lambda x: (x[0], x[1][1][0], x[1][1][1]))\
        .take(1)\

#.takeOrdered(1, key=lambda x: -x[1])
mejor


[(100014, 'camila', 'jasto')]

In [ ]:
# rdd productos de ventas (fecha, codigo prod, precio) otro rdd detalles
# (codigo prod, descripcion, categoria)
# cual es prod mas vendido
# cual es la categoria con mas productos vendidos
# cual es el top5 de productos mas vendidos (cod prod, desc, cant ventas)
# idem anterior calculando la categoria del productos que registro 
# mayor variacion de precios en el ultimo año

In [96]:
prods = [
    (20200201, 1, 40),
    (20190201, 1, 50),
    (20200201, 2, 120),
    (20190201, 2, 250),
    (20200201, 2, 110),
    (20200201, 3, 30),
    (20190201, 3, 20),
    (20190201, 3, 20),
    (20200201, 3, 60),
    (20200201, 4, 40),
    (20190201, 4, 20),
    (20200201, 5, 210),
    (20200201, 5, 210),
]
descr = [
    (1, "des", 1),
    (2, "des", 1),
    (3, "des", 2),
    (4, "des", 2),
    (5, "des", 2),
]

pro =  sc.parallelize(prods)
des = sc.parallelize(descr)

In [65]:
mas_ventas = pro.map(lambda x: (x[1], 1))\
                .reduceByKey(lambda x,y: x+y)\
                
mas_ventas.takeOrdered(1, key=lambda x: -x[1])

[(3, 4)]

In [69]:
des_cat = des.map(lambda x: (x[0], x[2]))

cat_mas_ventas = mas_ventas.join(des_cat)\
                    .map(lambda x: (x[1][1], x[1][0]))\
                    .reduceByKey(lambda x,y: x+y)\
                    .takeOrdered(1, key=lambda x: -x[1])
                         
cat_mas_ventas                

[(2, 8)]

In [92]:
def map_anio(x):
    if x>= 20200101:
        return 2020
    if x<20200101 and x>=20190101:
        return 2019

In [100]:
def filter_(x):
    if len(x) < 2:
        return False
    an19 = x[0]
    an20 = x[1]
    if an19[1] > an20[1]:
        return False
    return True

In [101]:
aumento_precio = pro.map(lambda x: ((x[1], map_anio(x[0])), x[2]))\
                .reduceByKey(lambda x,y:x+y)\
                .map(lambda x: (x[0][0], (x[0][1],x[1])))\
                .groupByKey()\
                .map(lambda x: (x[0], sorted(list(x[1]))))\
                .filter(lambda x: filter_(x[1]))\
                .takeOrdered(1, key=lambda x: -x[1][1][1])
aumento_precio

[(3, [(2019, 40), (2020, 90)])]